<h1 align='center'>BitCoin Data Transformation Internal</h1>

In [8]:
import pymongo
from pymongo import MongoClient
import matplotlib.pyplot as plt

client = MongoClient("129.26.78.40:27017")
db = client['BitCoin']

In [5]:
for a in db['senders_2017_week_3'].find().limit(10):
    print a
    print "\n"

{u'ntx': 1864, u'_id': None, u'tx': [{u'a': 1283613081, u'h': u'000000000000000000cb138683d669b655dd1c3e99f5b7fdc4012fef13bc38a5', u'r': u'152f1muMCNa7goXYhYAQC61hxEgGacmncB', u'ts': 1485043094, u't': datetime.datetime(2017, 1, 21, 23, 58, 14)}, {u'a': 1321966537, u'h': u'00000000000000000062bde54e63d108ffbf3c138431add696fc6c2da1aa7f1a', u'r': u'1KFHE7w8BhaENAswwryaoccDb6qcT6DbYY', u'ts': 1485036289, u't': datetime.datetime(2017, 1, 21, 22, 4, 49)}, {u'a': 1312292607, u'h': u'0000000000000000032392d5b48b85c88c6ecd4557ca93b1a8c6ecd824778660', u'r': u'152f1muMCNa7goXYhYAQC61hxEgGacmncB', u'ts': 1485039270, u't': datetime.datetime(2017, 1, 21, 22, 54, 30)}, {u'a': 1335107858, u'h': u'00000000000000000212acca6d47d1f486fcd6598b5b5ceb0b3356909594149a', u'r': u'1CK6KHY6MHgYvmRQ4PAafKYDrg1ejbH1cE', u'ts': 1485032913, u't': datetime.datetime(2017, 1, 21, 21, 8, 33)}, {u'a': 1255871015, u'h': u'000000000000000000cad22a1e22914593ed4dece2b5b6c426385d7f7d9ebd5f', u'r': u'1AZ6BkCo4zgTuuLpRStJH8iNseh

## Aggregate the BitCoin amount per transaction per address

In [ ]:
for i in range(1, 17):
    db["blocks_{0}".format(i)].aggregate([{"$project":
                                      {"block_index":"$block_index",
                                       "_id":0,
                                       "height":"$block_height",
                                       "relayed_by":"$relayed_by",
                                       "hash": "$hash",
                                       "tx":"$tx"}},
                                 {"$unwind":"$tx"},
                                 {"$project":
                                         {"inputs":"$tx.inputs",
                                          "out":"$tx.out",
                                          "time":"$tx.time",
                                          "hash": "$hash",
                                          "_id":0
                                         }},
                                 {"$unwind":"$inputs"},
                                 {"$project":
                                            {"time":"$time",
                                             "hash": "$hash",
                                             "address_source":"$inputs.prev_out.addr",
                                             "amount":"$inputs.prev_out.value",
                                             "out":"$out"}},
                                 {"$unwind":"$out"},                             
                                 {"$group": 
                                          {
                                            "_id": {"tx_hash": "$hash", "sender":"$address_source", "time": "$time", "reciever":"$out.addr", "tag":"$out.addr_tag_link"},                                                                                     
                                                "amount":{ "$sum": "$out.value"}                                   
                                                
                                            }
                                 },
                                {"$match": { "amount": {"$gt": 0}}},                                
                                {"$out":"blocks_{0}_aggregation".format(i)}],allowDiskUse=True)

## Collect all aggrefation into one collection

In [ ]:
for i in range(1, 13):
    collection = "blocks_{0}_aggregation".format(i)
    data = db[collection].find({})
    db["blocks_aggregation"].insert_many(data)

In [7]:
2600000./(24*7.*60)

257.93650793650795

## Group Data Hourly

In [90]:
for i in range(1, 13):
    db["blocks_{0}_aggregation".format(i)].aggregate([
                                 {"$group": {
                                     "_id": {"year": {"$year": "$tx_time" }, 
                                             "month": {"$month": "$tx_time"}, 
                                             "week": {"$week": "$tx_time"}, 
                                             "day": {"$dayOfMonth": "$tx_time"}, 
                                             "hour": {"$hour": "$tx_time"},
                                             "minutes": {"$minute":"$tx_time"},
                                             "second": {"$second":"$tx_time"}},
                                     "txs": {"$addToSet": {
                                            "h":"$_id.tx_hash",
                                            "s": "$_id.sender",
                                            "r": "$_id.reciever",
                                            "a": "$amount",                                          
                                            "t": "$tx_time"
                                     }}
                                 }},
                                
                                 {"$out":"blocks_{0}_aggregation_hourly".format(i)}],allowDiskUse=True)

OperationFailure: insert for $out failed: { connectionId: 183, err: "BSONObj size: 19116450 (0x123B1A2) is invalid. Size must be between 0 and 16793600(16MB) First element: _id: { week: 4, hour: 20, month: 1, second: 28...", code: 10334, n: 0, ok: 1.0 }

## Split the senders weekly

In [ ]:
db["blocks_aggregation"].aggregate([ {"$match": { "_id.time": {"$gt": mktime(datetime(2017, 1, 14).timetuple()), "$lte": mktime(datetime(2017, 1, 21).timetuple())}}},
                    {"$group": { 
                                 "_id": "$_id.sender", 
                                  "tx": {"$addToSet": {"hash":"$_id.tx_hash",
                                                       "receiver": "$_id.reciever",
                                                       "time": "$_id.time",
                                                       "amount": "$amount"}
                                        },
                                   "ntx": {"$sum": 1}
                                            }
                                },
                              {"$out":"senders_week_3"}
                             ],
                        allowDiskUse=True)

In [ ]:
print a